# 

# spatial.GeometryFM2D

``` python
spatial.GeometryFM2D(
    self,
    node_coordinates,
    element_table,
    codes=None,
    projection='LONG/LAT',
    dfsu_type=DfsuFileType.Dfsu2D,
    element_ids=None,
    node_ids=None,
    validate=True,
    reindex=False,
)
```

Flexible 2d mesh geometry.

## Attributes

| Name | Description |
|------------------------------------|------------------------------------|
| [boundary_codes](#mikeio.spatial.GeometryFM2D.boundary_codes) | Unique list of boundary codes. |
| [boundary_polygons](#mikeio.spatial.GeometryFM2D.boundary_polygons) | Lists of polygons defining domain outline. |
| [codes](#mikeio.spatial.GeometryFM2D.codes) | Node codes of all nodes (0=water, 1=land, 2…=open boundaries). |
| [element_coordinates](#mikeio.spatial.GeometryFM2D.element_coordinates) | Center coordinates of each element. |
| [is_2d](#mikeio.spatial.GeometryFM2D.is_2d) | Type is either mesh or Dfsu2D (2 horizontal dimensions). |
| [is_geo](#mikeio.spatial.GeometryFM2D.is_geo) | Are coordinates geographical (LONG/LAT)? |
| [is_layered](#mikeio.spatial.GeometryFM2D.is_layered) | Type is layered dfsu (3d, vertical profile or vertical column). |
| [is_local_coordinates](#mikeio.spatial.GeometryFM2D.is_local_coordinates) | Are coordinates relative (NON-UTM)? |
| [is_spectral](#mikeio.spatial.GeometryFM2D.is_spectral) | Type is spectral dfsu (point, line or area spectrum). |
| [is_tri_only](#mikeio.spatial.GeometryFM2D.is_tri_only) | Does the mesh consist of triangles only. |
| [max_nodes_per_element](#mikeio.spatial.GeometryFM2D.max_nodes_per_element) | The maximum number of nodes for an element. |
| [n_elements](#mikeio.spatial.GeometryFM2D.n_elements) | Number of elements. |
| [n_nodes](#mikeio.spatial.GeometryFM2D.n_nodes) | Number of nodes. |
| [projection](#mikeio.spatial.GeometryFM2D.projection) | The projection. |
| [projection_string](#mikeio.spatial.GeometryFM2D.projection_string) | The projection string. |

## Methods

| Name | Description |
|------------------------------------|------------------------------------|
| [contains](#mikeio.spatial.GeometryFM2D.contains) | Test if a list of points are contained by mesh. |
| [find_index](#mikeio.spatial.GeometryFM2D.find_index) | Find a *set* of element indicies for a number of points or within an area. |
| [find_nearest_elements](#mikeio.spatial.GeometryFM2D.find_nearest_elements) | Find index of nearest elements (optionally for a list). |
| [get_2d_interpolant](#mikeio.spatial.GeometryFM2D.get_2d_interpolant) | IDW interpolant for list of coordinates. |
| [get_element_area](#mikeio.spatial.GeometryFM2D.get_element_area) | Calculate the horizontal area of each element. |
| [get_node_centered_data](#mikeio.spatial.GeometryFM2D.get_node_centered_data) | Convert cell-centered data to node-centered by pseudo-laplacian method. |
| [get_overset_grid](#mikeio.spatial.GeometryFM2D.get_overset_grid) | Get a 2d grid that covers the domain by specifying spacing or shape. |
| [isel](#mikeio.spatial.GeometryFM2D.isel) | Export a selection of elements to a new geometry. |
| [to_mesh](#mikeio.spatial.GeometryFM2D.to_mesh) | Export geometry to new mesh file. |
| [to_shapely](#mikeio.spatial.GeometryFM2D.to_shapely) | Export mesh as shapely MultiPolygon. |

### contains

``` python
spatial.GeometryFM2D.contains(points)
```

Test if a list of points are contained by mesh.

#### Parameters

| Name | Type | Description | Default |
|--------|-----------------|------------------------------------|-----------|
| points | array-like n-by-2 | x,y-coordinates of n points to be tested | *required* |

#### Returns

| Name | Type       | Description                             |
|------|------------|-----------------------------------------|
|      | bool array | True for points inside, False otherwise |

### find_index

``` python
spatial.GeometryFM2D.find_index(x=None, y=None, coords=None, area=None)
```

Find a *set* of element indicies for a number of points or within an
area.

The returned indices returned are the unique, unordered set of element
indices that contain the points or area.

This method will return elements *containing* the argument points/area,
which is not necessarily the same as the nearest.

Typically not called directly, but by Dataset/DataArray’s sel() method.

#### Parameters

| Name | Type | Description | Default |
|----|-----------|----------------------------------------------------|-----|
| x | float \| np.ndarray \| None | X coordinate(s) (easting or longitude) | `None` |
| y | float \| np.ndarray \| None | Y coordinate(s) (northing or latitude) | `None` |
| coords | np.array(float, float) | As an alternative to specifying x, and y individually, the argument coords can be used instead. (x,y)-coordinates of points to be found, by default None | `None` |
| area | (float, float, float, float) | Bounding box of coordinates (left lower and right upper) to be selected, by default None | `None` |

#### Returns

| Name | Type     | Description                     |
|------|----------|---------------------------------|
|      | np.array | indicies of containing elements |

#### Raises

| Name | Type       | Description                        |
|------|------------|------------------------------------|
|      | ValueError | if any point is outside the domain |

#### Examples

``` python
>>> g = dfs.geometry
>>> id = dfs.find_index(x=3.1, y=4.3)
```

#### See Also

isel : get subset geometry for specific indicies find_nearest_elements :
find nearest instead of containing elements

### find_nearest_elements

``` python
spatial.GeometryFM2D.find_nearest_elements(
    x,
    y=None,
    n_nearest=1,
    return_distances=False,
)
```

Find index of nearest elements (optionally for a list).

#### Parameters

| Name | Type | Description | Default |
|----------|---------------|-----------------------------------------|-------|
| x | float \| np.ndarray | X coordinate(s) (easting or longitude) | *required* |
| y | float \| np.ndarray \| None | Y coordinate(s) (northing or latitude) | `None` |
| n_nearest | int | return this many (horizontally) nearest points for each coordinate set, default=1 | `1` |
| return_distances | bool | should the horizontal distances to each point be returned? default=False | `False` |

#### Returns

| Name | Type                 | Description                       |
|------|----------------------|-----------------------------------|
|      | np.array             | element ids of nearest element(s) |
|      | (np.array, optional) | horizontal distances              |

#### Examples

``` python
>>> g = dfs.geometry
>>> id = g.find_nearest_elements(3, 4)
>>> ids = g.find_nearest_elements([3, 8], [4, 6])
>>> ids = g.find_nearest_elements(xy)
>>> ids = g.find_nearest_elements(3, 4, n_nearest=4)
>>> ids, d = g.find_nearest_elements(xy, return_distances=True)
```

#### See Also

find_index : find element indicies for points or an area

### get_2d_interpolant

``` python
spatial.GeometryFM2D.get_2d_interpolant(
    xy,
    n_nearest=5,
    extrapolate=False,
    p=2,
    radius=None,
)
```

IDW interpolant for list of coordinates.

#### Parameters

| Name | Type | Description | Default |
|---------|----------|---------------------------------------------|---------|
| xy | array - like | x,y coordinates of new points | *required* |
| n_nearest | int | number of nearest elements used for IDW, by default 5 | `5` |
| extrapolate | bool | allow extrapolation, by default False | `False` |
| p | float | power of inverse distance weighting, default=2 | `2` |
| radius | float \| None | an alternative to extrapolate=False, only include elements within radius | `None` |

#### Returns

| Name | Type        | Description             |
|------|-------------|-------------------------|
|      | Interpolant | element ids and weights |

### get_element_area

``` python
spatial.GeometryFM2D.get_element_area()
```

Calculate the horizontal area of each element.

#### Returns

| Name | Type            | Description |
|------|-----------------|-------------|
|      | np.array(float) | areas in m2 |

### get_node_centered_data

``` python
spatial.GeometryFM2D.get_node_centered_data(data, extrapolate=True)
```

Convert cell-centered data to node-centered by pseudo-laplacian method.

#### Parameters

| Name | Type | Description | Default |
|-----------|--------------|-------------------------------------|-----------|
| data | np.array(float) | cell-centered data | *required* |
| extrapolate | bool | allow the method to extrapolate, default:True | `True` |

#### Returns

| Name | Type            | Description        |
|------|-----------------|--------------------|
|      | np.array(float) | node-centered data |

### get_overset_grid

``` python
spatial.GeometryFM2D.get_overset_grid(
    dx=None,
    dy=None,
    nx=None,
    ny=None,
    buffer=0.0,
)
```

Get a 2d grid that covers the domain by specifying spacing or shape.

#### Parameters

| Name | Type | Description | Default |
|-----|-----------|---------------------------------------------------|------|
| dx | float or (float, float) | grid resolution in x-direction (or in x- and y-direction) | `None` |
| dy | float | grid resolution in y-direction | `None` |
| nx | int | number of points in x-direction, by default None, (the value will be inferred) | `None` |
| ny | int | number of points in y-direction, by default None, (the value will be inferred) | `None` |
| buffer | float | positive to make the area larger, default=0 can be set to a small negative value to avoid NaN values all around the domain. | `0.0` |

#### Returns

| Name | Type              | Description |
|------|-------------------|-------------|
|      | \<mikeio.Grid2D\> | 2d grid     |

### isel

``` python
spatial.GeometryFM2D.isel(idx, keepdims=False, **kwargs)
```

Export a selection of elements to a new geometry.

Typically not called directly, but by Dataset/DataArray’s isel() or
sel() methods.

#### Parameters

| Name | Type | Description | Default |
|-----|-----|---------------------------------------------------------|------|
| idx | list(int) | collection of element indicies | *required* |
| keepdims | bool | Should the original Geometry type be kept (keepdims=True) or should it be reduced e.g. to a GeometryPoint2D if possible (keepdims=False), by default False | `False` |
| \*\*kwargs | Any | Not used | `{}` |

#### Returns

| Name | Type     | Description     |
|------|----------|-----------------|
|      | Geometry | geometry subset |

#### See Also

find_index : find element indicies for points or an area

### to_mesh

``` python
spatial.GeometryFM2D.to_mesh(outfilename)
```

Export geometry to new mesh file.

#### Parameters

| Name        | Type | Description                | Default    |
|-------------|------|----------------------------|------------|
| outfilename | str  | path to file to be written | *required* |

### to_shapely

``` python
spatial.GeometryFM2D.to_shapely()
```

Export mesh as shapely MultiPolygon.

#### Returns

| Name | Type                          | Description                 |
|------|-------------------------------|-----------------------------|
|      | shapely.geometry.MultiPolygon | polygons with mesh elements |